In [1]:
# test cuda availability
import torch
torch.cuda.is_available(), torch.cuda.get_device_name()

(True, 'Tesla T4')

In [2]:
# clone project
!git clone https://github.com/JennyGVoice/Language_Model.git

Cloning into 'Language_Model'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 46 (delta 16), reused 39 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 436.33 KiB | 17.45 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
%cd Language_Model
!pwd
!ls -a

/content/Language_Model
/content/Language_Model
.   .git	miniproject2_language_model.ipynb  src
..  main.ipynb	requirments.txt


In [ ]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!python -m src.train

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [ ]:
import torch
from src.dataset import CharDataset
from src.model.gpt_like import GPTLanguageModel

# load data
with open("data/input.txt", "r") as f:
    text = f.read()

dataset = CharDataset(text, block_size=128, split="train")

# choose checkpoint
ckpt_path = "experiments/checkpoints/step_200.pt"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = GPTLanguageModel(dataset.vocab_size).to(device)

# load checkpoint
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt["model"])
model.eval()

# test generation
prompt = "O God, O God!"
idx = torch.tensor([dataset.encode(prompt)]).to(device)
generated = model.generate(idx, max_new_tokens=300)
print(dataset.decode(generated[0].tolist()))

In [ ]:
from torch.utils.data import DataLoader
from src.model.metrics import calculate_ppl

val_dataset = CharDataset(text, block_size=128, split="val")
val_loader = DataLoader(val_dataset, batch_size=64)

ppl = calculate_ppl(model, val_loader, device=device)
print("Validation Perplexity:", ppl)